In [27]:
import numpy as np
import qutip as qt

## The Choi matrix representation of a single-qubit quantum channel $\Lambda: \mathcal{H_A} \rightarrow \mathcal{H_B}$ is obtained by applying the channel on a subsystem of the two-qubit maximally entangled state (i.e.~Bell states)
\begin{equation}
\begin{split}
\rho_{choi}&= (\mathbb{1} \otimes \Lambda) (|\phi^+\rangle \langle\phi^+|) \\ &=\sum_{i, j}|i\rangle\langle j| \otimes \Lambda(|i\rangle\langle j|).
\end{split}
\end{equation}

In [55]:
#Define the Bell state, the bit-flip gate and the Identity matrix

Bell=qt.bell_state(state='00')*qt.bell_state(state='00').dag()
#Bell=0.5*qt.Qobj([[1,0,0,1],[0,0,0,0],[0,0,0,0],[1,0,0,1]],dims = [[2, 2], [2, 2]])

X=qt.sigmax()
I=qt.qeye(2)

In [58]:
#Apply the bit-flip gate on a subsystem of the Bell state

Choi_X=qt.tensor(I,X)*Bell*qt.tensor(I,X).dag()
Choi_X

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0.  0.  0.  0. ]
 [0.  0.5 0.5 0. ]
 [0.  0.5 0.5 0. ]
 [0.  0.  0.  0. ]]

## Another approach which I consider it to be more general and practical way to obtain the Choi-matrix of an $n$-qubit channel is by considering the Kraus operators ${K_l}$ of the channel
\begin{equation}
\begin{split}
\rho_{choi}&=\sum_{i, j}|i\rangle\langle j| \otimes \left( \sum_{l}^r (K_l|i\rangle\langle j|K_l^\dagger) \right) \\ &= \sum_{l}^r |K_l\rangle\rangle \langle\langle K_l|,
\end{split}
\end{equation}

In [62]:
#vectorizing the gate's matrix by flattening it
vecX=qt.operator_to_vector(X)
#vecX=X.full().flatten()
vecX

Quantum object: dims = [[[2], [2]], [1]], shape = (4, 1), type = operator-ket
Qobj data =
[[0.]
 [1.]
 [1.]
 [0.]]

In [65]:
#Multiplying the vectorized gate with its conjugate-trasnposed self
Choi_X=vecX*vecX.dag()
#Choi_X=qt.Qobj(0.5*np.kron(X.full().flatten().reshape(4,1),X.full().flatten()),dims = [[2, 2], [2, 2]])
Choi_X

Quantum object: dims = [[[2], [2]], [[2], [2]]], shape = (4, 4), type = super, isherm = True, superrep = None
Qobj data =
[[0. 0. 0. 0.]
 [0. 1. 1. 0.]
 [0. 1. 1. 0.]
 [0. 0. 0. 0.]]

## The evolution of any quantum state $\sigma$ with respect to the channel's Choi-matrix is then given by
\begin{equation}
\mathcal{E}(\sigma) = d_A \operatorname{Tr}_{A}\left[\rho_{choi}\left(\sigma^{T} \otimes \mathbb{1_B}\right)\right]
\end{equation}

In [73]:
#Define any quantum state, up-state for example
Upstate=qt.qstate('u')*qt.qstate('u').dag()
Upstate

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0. 0.]
 [0. 1.]]

In [78]:
#The evolution of the up-state under the Choi-matrix of a bit-flip is expected to give the down-state
Downstate=2*(Choi_X*(qt.tensor(Upstate.trans(),I))).ptrace(1)
Downstate

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[1. 0.]
 [0. 0.]]

## Decomposing a channel into its Kraus operators can be done by obtaining the vectorized eigenvectors of the Choi-matrix, reshaping them into a square matrix, and weighting them by their corresponding eigenvalues and the square root of the subsystem's dimension $\sqrt{d_A}$.

In [80]:
#Eigenvalues of the Choi-Matrix
Choi_X.eigenstates()[0]

array([0.00000000e+00, 0.00000000e+00, 5.55111512e-16, 1.00000000e+00])

In [82]:
#Eigenvectors of the Choi-Matrix
Choi_X.eigenstates()[1]

array([Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
       Qobj data =
       [[1.]
        [0.]
        [0.]
        [0.]]                                                             ,
       Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
       Qobj data =
       [[0.]
        [0.]
        [0.]
        [1.]]                                                             ,
       Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
       Qobj data =
       [[ 0.        ]
        [ 0.70710678]
        [-0.70710678]
        [ 0.        ]]                                                    ,
       Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
       Qobj data =
       [[0.        ]
        [0.70710678]
        [0.70710678]
        [0.        ]]                                                     ],
      dtype=object)

In [101]:
#The only eigenvector that survives is the one with the non-zero eigenvalue
eigvecX=qt.Qobj(np.sqrt(2)*Choi_X.eigenstates()[0][3]*Choi_X.eigenstates()[1][3]
                , dims = [[[2], [2]], [1]], shape = (4, 1), type = 'operator-ket')
eigvecX

Quantum object: dims = [[[2], [2]], [1]], shape = (4, 1), type = operator-ket
Qobj data =
[[0.]
 [1.]
 [1.]
 [0.]]

In [102]:
#Reshaping the vector into a square matrix, we get our gate back as a single Kraus operator
qt.vector_to_operator(eigvecX)

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0. 1.]
 [1. 0.]]